### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


## Player Count

* Display the total number of players


In [3]:
total_players = len(purchase_data["SN"].unique()) #this line calculates the total players using unique and len
total_players_df = pd.DataFrame({"Total Players": [total_players]}) #this line displays the data in a data frame
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
#unique items
unique_items = len(purchase_data["Item Name"].unique())

#average price
avg_price = purchase_data["Price"].mean()

#number of purchases
purchase = purchase_data["Purchase ID"].count()

#total revenue
revenue = purchase_data["Price"].sum()

#create data frame with pd.DataFrame
summary_df = pd.DataFrame(
    {"Number of Unique Items": [unique_items],
    "Average Price" : [avg_price],
    "Number of Purchases": [purchase],
    "Total Revenue" : [revenue]}
    )

#use the map function to change format
summary_df["Average Price"] = summary_df["Average Price"].map("${:.2f}".format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map("${:.2f}".format)

summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
Gender_groups = purchase_data.groupby(["Gender"])

#We want to avoid counting duplicate SNs, so we call all the unique SN's grouped by males and females. 
#We could have used .drop_duplicates
Gender_counts = Gender_groups["SN"].unique()


#Now we count
Male_Count = len(Gender_counts["Male"])
Female_Count = len(Gender_counts["Female"])
Other_Count = len(Gender_counts["Other / Non-Disclosed"])

#Arithmetic for percentages
Total_Count = Male_Count + Female_Count + Other_Count
Male_Perc = round(Male_Count/Total_Count*100,2)
Female_Perc = round(Female_Count/Total_Count*100,2)
Other_Perc = round(Other_Count/Total_Count*100,2)

#Create a dictionary to turn into a dataframe
Gender_dict = {
    "Total Count" : [Male_Count, Female_Count, Other_Count],
    "Percentage of Players" : [Male_Perc, Female_Perc, Other_Perc],
    " " : ["Male", "Female", "Other / Non Disclosed"]
}

Gender_Summary = pd.DataFrame(Gender_dict)

#Use the map method to add percentage signs
Gender_Summary["Percentage of Players"] = Gender_Summary["Percentage of Players"].map("{:}%".format)

#Finishing touches to make the table look cleaner
Gender_Summary.set_index(' ')

,Total Count,Percentage of Players
,,
Male,484,84.03%
Female,81,14.06%
Other / Non Disclosed,11,1.91%


In [6]:
#Another way of doing this using .loc

##
#Genderdf = pd.DataFrame(purchase_data[["Gender","SN"]])
#Genderdf

#filters the data frame for the specific gender, and then counts the Unique SNs in that gender
#males = Genderdf.loc[Genderdf["Gender"] == "Male", :]
#malescount = len(males["SN"].unique())


#females = Genderdf.loc[Genderdf["Gender"] == "Female",:]
#femcount = len(females["SN"].unique())

#undisc = Genderdf.loc[Genderdf["Gender"] == "Other / Non-Disclosed", :]
#undiscount = len(undisc["SN"].unique())

#maleperc = round(malescount/total_players*100,2)
#femperc = round(femcount/total_players*100,2)
#undiscperc = round(undiscount/total_players*100,2)



Gender_Summary

,Total Count,Percentage of Players,
0,484,84.03%,Male
1,81,14.06%,Female
2,11,1.91%,Other / Non Disclosed



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#Here I've taken the previous grouby object that was sorted by gender and got the Purchase count, avg price and total value.
#We turn them into data frames so that we may put a column name and append them
Purchase_Count = pd.DataFrame(Gender_groups["Purchase ID"].count())
Average_Price = pd.DataFrame(Gender_groups["Price"].mean())
Total_Purchase_Value = pd.DataFrame(Gender_groups["Price"].sum())

#Its a little different for the total per person due to the duplicates of 'SN' entries
#So we use the aggregate function to add values of the same SN name
Price_Gender = purchase_data.groupby(["SN"])

#include gender in the aggregates dict here so that we retain that information in the new data frame. 
#The SN:first aggregates all the duplicate SNs, with the Price:sum. Drop duplicates does not work 
aggregates = {"SN":"first","Gender": "first", "Price":"sum"}
Price_Gendernew = Price_Gender.aggregate(aggregates)

#Do another groupby, by gender to determine avg total per person
Avg_Tot_Per_Person = Price_Gendernew.groupby(["Gender"]).mean()


#Now we append columns from the different 
Purchase_Count["Average Purchase Price"] = Average_Price["Price"]
Purchase_Count["Total Purchase Value"] = Total_Purchase_Value["Price"]
Purchase_Count["Avg Total Purchase per Person"] = Avg_Tot_Per_Person["Price"]

#Rename Purchase Count
Purchase_Analysis = Purchase_Count
Purchase_Analysis

#use mapping to change format and rename columns
Purchase_Analysis = Purchase_Analysis.rename(columns={"Purchase ID": "Purchase Count"})
Purchase_Analysis["Average Purchase Price"] = Purchase_Analysis["Average Purchase Price"].map("${:.2f}".format)
Purchase_Analysis["Total Purchase Value"] = Purchase_Analysis["Total Purchase Value"].map("${:,.2f}".format)
Purchase_Analysis["Avg Total Purchase per Person"] = Purchase_Analysis["Avg Total Purchase per Person"].map("${:,.2f}".format)

#Output
Purchase_Analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
#Create the bins and labels

bins = [0,9,14,19,24,29,34,39,99999]
age = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#categorize and add a column showing their age range
purchase_data["Age group"] = pd.cut(purchase_data["Age"], bins, labels=age, include_lowest=True)

#Use the groupby to group them, and drop duplicate SNs
purchase_datanew = purchase_data.drop_duplicates(subset="SN")
Age_groups = purchase_datanew.groupby("Age group")

#turn it into a data frame in order to create a column name to work with
Age_Demo = pd.DataFrame(Age_groups["Age group"].count())

#Append a new column that displays percentages
Age_Demo["Percentage of Players"] = Age_Demo["Age group"]/total_players*100
Age_Demo = Age_Demo.rename(columns={"Age group" : "Total Count"})
Age_Demo["Percentage of Players"] = Age_Demo["Percentage of Players"].map("{:.2f}%".format)
Age_Demo

,Total Count,Percentage of Players
Age group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
#We have to groupby again
purchase_age = purchase_data.groupby("Age group")
Purc_Count = pd.DataFrame(purchase_age["SN"].count())
Avg_Price = pd.DataFrame(purchase_age["Price"].mean())
Total_Price = pd.DataFrame(purchase_age["Price"].sum())

#Aggregate the duplicate SNs, drop duplicates do not work
purchase_agenew = purchase_data.groupby("SN")
aggregates = {"Age group": "first", "SN":"first", "Price":"sum"}
purchase_agenew = purchase_agenew.aggregate(aggregates)
purchase_agenew = purchase_agenew.groupby("Age group")
AvgPer = pd.DataFrame(purchase_agenew["Price"].mean())

#Make the data frame by appending columns
Age_Analysis = Purc_Count
Age_Analysis = Age_Analysis.rename(columns={"SN":"Purchase Count"})
Age_Analysis["Average Purchase Price"] = Avg_Price["Price"]
Age_Analysis["Total Purchase Value"] = Total_Price["Price"]
Age_Analysis["Avg Total Purchase per Person"] = AvgPer["Price"]
Age_Analysis

#formatting
Age_Analysis["Average Purchase Price"] = Age_Analysis["Average Purchase Price"].map("${:.2f}".format)
Age_Analysis["Total Purchase Value"] = Age_Analysis["Total Purchase Value"].map("${:,.2f}".format)
Age_Analysis["Avg Total Purchase per Person"] = Age_Analysis["Avg Total Purchase per Person"].map("${:,.2f}".format)
Age_Analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
#Use groupby again and use value counts

#We extract the information that tells us how many times someone has spent in descending order using grouby and count
SpenderGroups = purchase_data.groupby("SN")
SN_Count = SpenderGroups["SN"].count()
SN_Count = pd.DataFrame(SN_Count)
SN_Count = SN_Count.rename(columns={"SN": "Purchase Count"})

Avg_Price = SpenderGroups["Price"].mean()
Avg_Price = pd.DataFrame(Avg_Price)
Avg_Price= Avg_Price.rename(columns={"Price":"Average Purchase Price"})

Total_Value = SpenderGroups["Price"].sum()
Total_Value = pd.DataFrame(Total_Value)
Total_Value = Total_Value.rename(columns={"Price":"Total Purchase Value"})

#Create the data frame

Top_Spenders = SN_Count
Top_Spenders["Average Purchase Price"] = Avg_Price["Average Purchase Price"]
Top_Spenders["Total Purchase Value"] = Total_Value["Total Purchase Value"]
Top_Spenders = Top_Spenders.sort_values("Total Purchase Value", ascending=False)

#formatting
Top_Spenders["Average Purchase Price"] = Top_Spenders["Average Purchase Price"].map("${:.2f}".format)
Top_Spenders["Total Purchase Value"] = Top_Spenders["Total Purchase Value"].map("${:,.2f}".format)
Top_Spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
#Call specific columns, The Id, Name and Price
Item_ID = purchase_data[["Item ID", "Item Name", "Price"]]

#Groupby to count and sum the prices by item and create them as data frames to combine later
grouped = Item_ID.groupby(["Item ID", "Item Name"])
PurchaseCount = pd.DataFrame(grouped["Price"].count())
PurchaseCount = PurchaseCount.rename(columns={"Price":"Purchase Count"})
TotalPurchase = pd.DataFrame(grouped["Price"].sum())
TotalPurchase = TotalPurchase.rename(columns={"Price":"Total Purchase Value"})
Prices = pd.DataFrame(grouped["Price"].mean())

#Combine the data frames

#Direct column adding from the previous parts do not work, merge works
Items = PurchaseCount
Items = pd.merge(PurchaseCount, Prices, on=["Item ID", "Item Name"])
Items = pd.merge(Items,TotalPurchase, on=["Item ID","Item Name"])

#Formatting 
PopularItems = Items.sort_values("Purchase Count", ascending=False).head()
PopularItems["Price"] = PopularItems["Price"].map("${:.2f}".format)
PopularItems["Total Purchase Value"] = PopularItems["Total Purchase Value"].map("${:,.2f}".format)
PopularItems = PopularItems.rename(columns={"Price":"Item Price"})
PopularItems

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [16]:
ProfitableItems = Items.sort_values("Total Purchase Value", ascending=False).head()
ProfitableItems["Price"] =ProfitableItems["Price"].map("${:.2f}".format)
ProfitableItems["Total Purchase Value"] = ProfitableItems["Total Purchase Value"].map("${:,.2f}".format)
ProfitableItems = ProfitableItems.rename(columns={"Price":"Item Price"})
ProfitableItems

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
